In [29]:
from bs4 import BeautifulSoup as bs
import requests
import nltk
from difflib import SequenceMatcher
import pandas as pd
from urllib.parse import quote
import time

def similarity(str1, str2):
    return (SequenceMatcher(None,str1,str2).ratio())*100
def getTextFromHtml(text):
  return text.replace('<strong>', '').replace('</strong>', '').replace('<cite>', '').replace('</cite>', '').replace('<p>', '').replace('</p>', '')
def querry(searchString, show_score=False):
  is_plagrism = False
  is_plagrism_links = []
  start = time.time()
  TIMEOUT_DURATION = 3
  PLAGIARISED_THRESHOLD = 0.8
  url = 'https://www.bing.com/search?q='+quote(searchString)
  print(url)
  try:
      content = requests.get(url, headers = {'User-agent': 'your bot 0.1'}, verify=False, timeout=TIMEOUT_DURATION).content
  except:
      return 'null'
  soup = bs(content, 'html.parser')
  # print(soup)
  body = soup.find('body')
  # print(body)
  no_result = body.find_all('li', {"class": "b_no"})
  for r in no_result:
    result_string = r.select('h1')
    if(str(result_string).startswith('[<h1>There are no results for')):
      return 'null'
  results = body.find_all('div', {"class": "b_caption"})
  for result in results:
    cite = result.find('cite')
    # print(result)
    text_para = result.find('p')
    # print(searchString)
    search_unique = set(searchString.lower().split(' '))
    # print(len(search_unique))

    # print(text_para.text)
    # if text_para is not None:
    result_text = set(text_para.text.split('·')[-1].strip().replace('...','').lower().split(' '))
  # print(len(result_text))
    bold_unique = set(getTextFromHtml(str(result.find_all('strong')).replace('[', '').replace(']', '').replace(',','')).lower().split(' '))
  # print(len(bold_unique))
  
    match_bold = len(search_unique&bold_unique)/len(search_unique)
    result_match = len(result_text&bold_unique)/len(bold_unique)
    if show_score:
        print(f'% match bold: {match_bold}')
        print(f'% result match: {result_match}')
    if match_bold > 0.5 and result_match > 0.5:
      is_plagrism = True
      is_plagrism_links.append(cite)
    print("-----"*3)
    # cite = result.find('cite')
    # content = result.select('p')
    # print("link: ", getTextFromHtml(str(cite)))
    # print("content: ", getTextFromHtml(str(content).replace('[', '').replace(']', '')))
    # highlighted_sentences = getTextFromHtml(str(content).replace('[', '').replace(']', '')).split(".")
    # print("bolded sentences: ", highlighted_sentences)
    # for sentence in highlighted_sentences:
    #   sentence = sentence.strip()
    #   if(sentence=='.' or len(sentence)==0):
    #     highlighted_sentences.remove(sentence)
    # print("post processed bolded sentences: ", highlighted_sentences)
    # try:
    #     website_content = requests.get(getTextFromHtml(str(cite)), timeout=TIMEOUT_DURATION)
    # except:
    #   continue
    # while True:
    #   website_content = website_content.content
    #   if(website_content!=''):
    #     break
    # website_text = bs(website_content, "lxml").text
    # print(website_text)
    # matched_sentences = []
  return is_plagrism,is_plagrism_links

searchString = 't '
def inference(searchString, showlink=False, show_score=False):
    is_plag, links = querry(searchString, show_score=show_score)
    if is_plag:
        print('Is Plagiarism')
    if showlink:
        print('Links:')
        for i in links:
            print('\t', i.text)
        else:
            print('Not Plagiarism')
    return is_plag


def inference_paragraph(search_paragraph):
  sents = nltk.sent_tokenize(search_paragraph)
  list_idx = []
  final_list = []
  for i,sent in enumerate(sents):
    is_plag = inference(sent)
    if is_plag:
      list_idx.append(i)
    final_list.append((sent, is_plag))
  return final_list


In [38]:
def demo_funtion(text):
    sent1 = 'The quick brown fox jumps over the lazy dog.'
    sent2 = 'The quick brown fox jumps over the lazy cog.'
    sent3 = 'The quick brown fox jumps over the lazy dog.'
    return [(sent1, "+"), (sent2, "-"), (sent3, "+")]

In [30]:
inference('Trước đó, trên trang Facebook cá nhân có hơn 28 nghìn lượt theo dõi của Chu Thanh Huyền (được nhiều người cho rằng là bạn gái cầu thủ Đội tuyển Quốc gia Việt Nam Nguyễn Quang Hải) đã đăng tải một bài viết kèm hình ảnh về việc bản thân và gia đình bị đe doạ hành hung, tạt axit.')

https://www.bing.com/search?q=Tr%C6%B0%E1%BB%9Bc%20%C4%91%C3%B3%2C%20tr%C3%AAn%20trang%20Facebook%20c%C3%A1%20nh%C3%A2n%20c%C3%B3%20h%C6%A1n%2028%20ngh%C3%ACn%20l%C6%B0%E1%BB%A3t%20theo%20d%C3%B5i%20c%E1%BB%A7a%20Chu%20Thanh%20Huy%E1%BB%81n%20%28%C4%91%C6%B0%E1%BB%A3c%20nhi%E1%BB%81u%20ng%C6%B0%E1%BB%9Di%20cho%20r%E1%BA%B1ng%20l%C3%A0%20b%E1%BA%A1n%20g%C3%A1i%20c%E1%BA%A7u%20th%E1%BB%A7%20%C4%90%E1%BB%99i%20tuy%E1%BB%83n%20Qu%E1%BB%91c%20gia%20Vi%E1%BB%87t%20Nam%20Nguy%E1%BB%85n%20Quang%20H%E1%BA%A3i%29%20%C4%91%C3%A3%20%C4%91%C4%83ng%20t%E1%BA%A3i%20m%E1%BB%99t%20b%C3%A0i%20vi%E1%BA%BFt%20k%C3%A8m%20h%C3%ACnh%20%E1%BA%A3nh%20v%E1%BB%81%20vi%E1%BB%87c%20b%E1%BA%A3n%20th%C3%A2n%20v%C3%A0%20gia%20%C4%91%C3%ACnh%20b%E1%BB%8B%20%C4%91e%20do%E1%BA%A1%20h%C3%A0nh%20hung%2C%20t%E1%BA%A1t%20axit.


C:\Users\quangvh3\Miniconda3\envs\myenv\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


---------------
---------------


False

In [27]:
inference_paragraph(default_para)

https://www.bing.com/search?q=Tr%C6%B0%E1%BB%9Bc%20%C4%91%C3%B3%2C%20tr%C3%AAn%20trang%20Facebook%20c%C3%A1%20nh%C3%A2n%20c%C3%B3%20h%C6%A1n%2028%20ngh%C3%ACn%20l%C6%B0%E1%BB%A3t%20theo%20d%C3%B5i%20c%E1%BB%A7a%20Chu%20Thanh%20Huy%E1%BB%81n%20%28%C4%91%C6%B0%E1%BB%A3c%20nhi%E1%BB%81u%20ng%C6%B0%E1%BB%9Di%20cho%20r%E1%BA%B1ng%20l%C3%A0%20b%E1%BA%A1n%20g%C3%A1i%20c%E1%BA%A7u%20th%E1%BB%A7%20%C4%90%E1%BB%99i%20tuy%E1%BB%83n%20Qu%E1%BB%91c%20gia%20Vi%E1%BB%87t%20Nam%20Nguy%E1%BB%85n%20Quang%20H%E1%BA%A3i%29%20%C4%91%C3%A3%20%C4%91%C4%83ng%20t%E1%BA%A3i%20m%E1%BB%99t%20b%C3%A0i%20vi%E1%BA%BFt%20k%C3%A8m%20h%C3%ACnh%20%E1%BA%A3nh%20v%E1%BB%81%20vi%E1%BB%87c%20b%E1%BA%A3n%20th%C3%A2n%20v%C3%A0%20gia%20%C4%91%C3%ACnh%20b%E1%BB%8B%20%C4%91e%20do%E1%BA%A1%20h%C3%A0nh%20hung%2C%20t%E1%BA%A1t%20axit.


C:\Users\quangvh3\Miniconda3\envs\myenv\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


---------------
---------------
---------------
---------------
---------------
---------------
https://www.bing.com/search?q=Th%E1%BA%ADm%20ch%C3%AD%2C%20c%C3%B4%20g%C3%A1i%20c%C3%B2n%20b%E1%BB%8B%20%C4%91e%20do%E1%BA%A1%20l%C3%A0m%20lan%20truy%E1%BB%81n%20c%C3%A1c%20h%C3%ACnh%20%E1%BA%A3nh%20nh%E1%BA%A1y%20c%E1%BA%A3m%20tr%C3%AAn%20danh%20ngh%C4%A9a%20c%E1%BB%A7a%20Huy%E1%BB%81n.


C:\Users\quangvh3\Miniconda3\envs\myenv\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
https://www.bing.com/search?q=Theo%20nh%E1%BB%AFng%20h%C3%ACnh%20%E1%BA%A3nh%20ch%E1%BB%A5p%20l%E1%BA%A1i%20%C4%91o%E1%BA%A1n%20tin%20nh%E1%BA%AFn%2C%20c%C3%B3%20m%E1%BB%99t%20ng%C6%B0%E1%BB%9Di%20t%E1%BB%B1%20x%C6%B0ng%20l%C3%A0%20b%C3%AAn%20th%E1%BB%A9%203%20%C4%91%C6%B0%E1%BB%A3c%20thu%C3%AA%20%C4%91%E1%BB%83%20t%E1%BA%A5n%20c%C3%B4ng%20trang%20c%C3%A1%20nh%C3%A2n%20v%C3%A0%20c%C3%A1c%20t%C3%A0i%20kho%E1%BA%A3n%20m%E1%BA%A1ng%20x%C3%A3%20h%E1%BB%99i%20c%E1%BB%A7a%20Huy%E1%BB%81n.


C:\Users\quangvh3\Miniconda3\envs\myenv\lib\site-packages\urllib3\connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------


[('Trước đó, trên trang Facebook cá nhân có hơn 28 nghìn lượt theo dõi của Chu Thanh Huyền (được nhiều người cho rằng là bạn gái cầu thủ Đội tuyển Quốc gia Việt Nam Nguyễn Quang Hải) đã đăng tải một bài viết kèm hình ảnh về việc bản thân và gia đình bị đe doạ hành hung, tạt axit.',
  False),
 ('Thậm chí, cô gái còn bị đe doạ làm lan truyền các hình ảnh nhạy cảm trên danh nghĩa của Huyền.',
  False),
 ('Theo những hình ảnh chụp lại đoạn tin nhắn, có một người tự xưng là bên thứ 3 được thuê để tấn công trang cá nhân và các tài khoản mạng xã hội của Huyền.',
  False)]

In [39]:
demo_funtion("asdfafs")

[('The quick brown fox jumps over the lazy dog.', '+'),
 ('The quick brown fox jumps over the lazy cog.', '-'),
 ('The quick brown fox jumps over the lazy dog.', '+')]

In [40]:

import gradio
# https://gradio.app/docs/#interface

#interface congifuration
fn=inference_paragraph

default_para = """Trước đó, trên trang Facebook cá nhân có hơn 28 nghìn lượt theo dõi của Chu Thanh Huyền (được nhiều người cho rằng là bạn gái cầu thủ Đội tuyển Quốc gia Việt Nam Nguyễn Quang Hải) đã đăng tải một bài viết kèm hình ảnh về việc bản thân và gia đình bị đe doạ hành hung, tạt axit. Thậm chí, cô gái còn bị đe doạ làm lan truyền các hình ảnh nhạy cảm trên danh nghĩa của Huyền.


Theo những hình ảnh chụp lại đoạn tin nhắn, có một người tự xưng là bên thứ 3 được thuê để tấn công trang cá nhân và các tài khoản mạng xã hội của Huyền. 

"""
inputs=gradio.inputs.Textbox( lines=17, placeholder="Put your writing here", default=default_para, label=None)
color_map={"+": "lightgreen", "-": "pink"}
outputs=gradio.outputs.HighlightedText(color_map=None, label=None)
# outputs=gradio.outputs.Textbox(type="auto", label=None)



enable_queue = False

iface_detection = gradio.Interface(fn=demo_funtion, inputs=inputs, outputs=outputs, verbose=False, examples=None, examples_per_page=10,
                     live=False, layout="unaligned", capture_session=False, interpretation=None, num_shap=2.0,
                     theme=None, title=None, description=None, article=None, thumbnail=None,
                     css=None, server_port=None, server_name="127.0.0.1", allow_screenshot=True,
                     allow_flagging=None, flagging_options=None, encrypt=False, show_tips=False,
                     flagging_dir="flagged", enable_queue=enable_queue, api_mode=False)

#launch config
share=False
# auth=[("quang", "quang"), ("quang1", "quang1")]
auth = None # no auth
iface_detection.launch(inline=None, inbrowser=None, share=share, debug=False, auth=auth, auth_message=None)


Running on local URL:  http://127.0.0.1:7871/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7871/', None)

In [59]:

# https://gradio.app/docs/#interface

#interface congifuration
def FunctionName(a, b):
    return a + b
    
fn=FunctionName
inputs=None
outputs=gradio.outputs.Textbox()
enable_queue = False

iface_generator = gradio.Interface(fn=FunctionName, inputs=input, outputs=outputs)

#launch config
share=False
# auth=[("quang", "quang"), ("quang1", "quang1")]
auth = None # no auth
iface_generator.launch(inline=None, inbrowser=None, share=share, debug=False, auth=auth, auth_message=None)


Running on local URL:  http://127.0.0.1:7878/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7878/', None)

In [53]:
input

[CheckboxGroup(label="None"), CheckboxGroup(label="None")]

In [48]:
input

[CheckboxGroup(label="None"), CheckboxGroup(label="None")]

In [60]:
demo_detector_output

[('The quick brown fox jumps over the lazy dog.', '+'),
 ('The quick brown fox jumps over the lazy cog.', '-'),
 ('The quick brown fox jumps over the lazy dog.', '+')]

In [58]:
demo_detector_output= demo_funtion("asdfasf")
input=[]
for sent in demo_detector_output:
    if sent[-1]=="+":
        input.append(gradio.inputs.Dropdown(["ran", "swam", "ate", "slept"], label=sent))



In [42]:


gen_sentences = [["sent1", "sent2", "sent3"],
             ["sent1", "sent2", "sent3"],
             ["sent1", "sent2", "sent3"]]
input = [gradio.inputs.Dropdown(gen_sentences[i]) for i in range(len(gen_sentences))]
gen_sents = [("facebook", ["sent1", "sent2", "sent3"]),
             ("uyhsda", ["sent1", "sent2", "sent3"]),
             ("asdfsad", ["sent1", "sent2", "sent3"])]
inputs = []
for i in range(len(gen_sents)):
    choices = gen_sents[i][1]
    label = gen_sents[i][0]
    inputs.append(gradio.inputs.Dropdown(choices, label=label))

gen_sentences = [["sent1", "sent2", "sent3"],
             ["sent1", "sent2", "sent3"],
             ["sent1", "sent2", "sent3"]]
input = [gradio.inputs.Dropdown(gen_sentences[i]) for i in range(len(gen_sentences))]


# iface = gradio.Interface(
#     sentence_builder,
#     [
#         gradio.inputs.Slider(2, 20),
#         gradio.inputs.Dropdown(["cats", "dogaaaaaaaaaaaaaaaaaaa", "biraaaaaaaaaaaaaaaaaaaaaaaad"]),
#         gradio.inputs.Radio(["park", "zoo", "road"]),
#         gradio.inputs.Checkboxgradiooup(["ran", "swam", "ate", "slept"]),
#         gradio.inputs.Checkbox(label="Is it the morning?"),
#     ],
#     "text",
#     examples=[
#         [2, "cat", "park", ["ran", "swam"], True],
#         [4, "dog", "zoo", ["ate", "swam"], False],
#         [10, "bird", "road", ["ran"], False],
#         [8, "cat", "zoo", ["ate"], True],
#     ],
# )

input = [gradio.inputs.Dropdown(gen_sentences[i]) for i in range(len(gen_sentences))]

iface = gradio.Interface(
    demo_funtion,
    [
        gradio.inputs.Slider(2, 20),
        gradio.inputs.Dropdown(["cats", "dogaaaaaaaaaaaaaaaaaaa", "biraaaaaaaaaaaaaaaaaaaaaaaad"]),
        gradio.inputs.Radio(["park", "zoo", "road"]),
        gradio.inputs.CheckboxGroup(["ran", "swam", "ate", "slept"]),
        gradio.inputs.Checkbox(label="Is it the morning?"),
    ],
    "text",
    examples=[
        [2, "cat", "park", ["ran", "swam"], True],
        [4, "dog", "zoo", ["ate", "swam"], False],
        [10, "bird", "road", ["ran"], False],
        [8, "cat", "zoo", ["ate"], True],
    ],
)

iface.launch()


Running on local URL:  http://127.0.0.1:7872/

To create a public link, set `share=True` in `launch()`.


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7872/', None)